# Data 3-4 Pandas Iterations and lambdas

Apply, lambdas, iterrows, itertuples

In this lesson we will start learning how to clean a dataframe data and loop over it

In [3]:
import pandas as pd

checks = pd.read_csv('https://raw.githubusercontent.com/mafudge/datasets/refs/heads/master/dining/check-data.csv')
checks.sample(10)

,check,date,party size,total items on check,total amount of check,gratuity
45,4031,2024-07-12,9,13,$129.74,$29.84
40,2512,2024-03-30,3,12,$181.56,$39.94
34,1368,2024-12-21,10,25,"$2,193.00",$372.81
1,2443,2024-06-09,3,10,$286.40,$31.50
37,4829,2024-12-30,9,11,$816.20,$16.32
25,4031,2024-08-12,6,14,$655.48,$65.55
31,1945,2024-02-05,3,7,$132.86,$21.26
14,3676,2024-02-25,1,1,$19.89,$1.99
18,2103,2024-02-24,8,9,$388.26,$116.48
23,1194,2024-07-06,2,6,$453.06,$72.49


In [4]:
# Note: numbers are not numbers!!!
checks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   check                  50 non-null     int64 
 1   date                   50 non-null     object
 2   party size             50 non-null     int64 
 3   total items on check   50 non-null     int64 
 4   total amount of check  50 non-null     object
 5   gratuity               50 non-null     object
dtypes: int64(3), object(3)
memory usage: 2.5+ KB


## Apply

Apply allows us to execute a function over a Series or the entire DataFrame.


`series.Apply(func)` <== call function `func` for every item in the Series

`dataframe.Apply(lambda row: func, axis=1)` <== call function `func` for every row in the DataFrame `axis=1` == row

`dataframe.Apply(lambda col: func, axis=0)` <== call function `func` for every row in the DataFrame `axis=0` == col


### Why Apply ?

Apply helps us clean up our data because we can execute non-trivial transformations over our dataframes.

For example, we want to enhance this data by calculating the `price per item` this is defined as:

`total amount of check` / `total items on check`

The problem is `total amount of check` is an `object`, not a `float`. This means we cannot do math on it.


In [5]:
# Type error because of the dollar sign and commas!!!
checks['price_per_item'] = checks['total amount of check'] / checks['total items on check']

TypeError: unsupported operand type(s) for /: 'str' and 'int'

How do we fix this? we write a user-defined function to convert string values like this: `$4,590.45` into floats like this: `4590.45`

In [6]:
def clean_currency(value:str) -> float:
    '''
    This function will take a string value and remove the dollar sign and commas
    and return a float value.
    '''
    return float(value.replace(',', '').replace('$', ''))


# tests
assert clean_currency('$1,000.00') == 1000.00
assert clean_currency('$1,000') == 1000.00
assert clean_currency('1,000') == 1000.00
assert clean_currency('$1000') == 1000.00

With our function written we can use `apply()` to transform the series.

Remember its a really good idea to **track lineage** when you are building a data pipeline. 

**NEVER** replace columns, always create new ones.

In [7]:
checks['total_amount_of_check_cleaned'] = checks['total amount of check'].apply(clean_currency)
checks['price_per_item'] = checks['total_amount_of_check_cleaned'] / checks['total items on check']
checks.sample(10)

,check,date,party size,total items on check,total amount of check,gratuity,total_amount_of_check_cleaned,price_per_item
29,4590,2024-05-08,3,5,$220.40,$22.04,220.40,44.08
33,3842,2024-03-31,6,6,$147.12,$5.88,147.12,24.52
37,4829,2024-12-30,9,11,$816.20,$16.32,816.20,74.20
49,3404,2024-07-19,9,26,"$2,382.90",$71.49,2382.90,91.65
43,1186,2024-09-21,5,16,$298.72,$74.68,298.72,18.67
28,2446,2024-12-15,4,12,$575.64,$28.78,575.64,47.97
9,2968,2024-12-28,1,3,$122.97,$23.36,122.97,40.99
0,2827,2024-05-06,8,12,$415.08,$107.92,415.08,34.59
36,3193,2024-05-12,2,5,$383.30,$111.16,383.30,76.66
34,1368,2024-12-21,10,25,"$2,193.00",$372.81,2193.00,87.72


In [9]:
def extract_year_ymd(date_in_ymd:str) -> int:
    '''
    This function will take a date string and return the year as an integer.
    '''
    return int(date_in_ymd.split('-')[0])

assert extract_year_ymd('2019-01-01') == 2019
assert extract_year_ymd('2020-12-31') == 2020


## Challenge 3-4-1

#### Modularize our work!

Let's take what we did so far, and create a dataset that would be better prepared for analysis / machine learning.

1. create a module `check_functions.py`
    - add the `clean_currency()` function definition to it.
    - under `if __name__=='__main__':` add the tests
    - run the code to make sure it works.
2. create your challenge file `3-4-1.py`
    - import streamlit, pandas and your clean_currency function
    - load the checks dataset into a dataframe: 
    - clean the `total amount of check` and `gratuity` columns
    - calculate the `price_per_item`  as total amount of check / total items on check
    - calcualte the `price_per_person` as total amont of check / party size
    - calcualte the `items_per_person` as total items on check / party size
    - calcualte the `tip_percentage` as the total amount of check / gratuity
    - display dataframe
    - describe dataframe
    


checks dataset `https://raw.githubusercontent.com/mafudge/datasets/refs/heads/master/dining/check-data.csv`



## Using Row Apply to setup some KPI's 

**KPI** is a key performance indicator. It summarizes larger points of data, so they can be measured over time. For example a letter grade such as an A- is a KPI summary of all your graded efforts to date.

- Let's build some simple KPIs from this data. 
- While KPI's are determined and decided upon from the business decision makers,
- Actionable KPI's are always based on data evidence.

**KPI 1: Whales**

With your help, marketing has decided that whale customers have checks that are:
    - `whale` whales are in the top 75% percentile for both items per person and price per person 
    - `big eaters` whales are in the top 75% for items per person 
    - `big spenders` are in the top 75% for price per person 

**KPI 2: Tippers**

With your help, marketing has decided that `light` tippers are in the botton 25% of tip percentage and `heavy` tippers are in the top 75% 

### Before we can apply our KPI's we must write the functions!

In [10]:
checks['gratuity_cleaned'] = checks['gratuity'].apply(clean_currency)
checks['price_per_item'] = checks['total_amount_of_check_cleaned'] / checks['total items on check']
checks['price_per_person'] = checks['total_amount_of_check_cleaned'] / checks['party size']
checks['items_per_person'] = checks['total items on check'] / checks['party size']
checks['tip_percentage'] = checks['gratuity_cleaned'] / checks['total_amount_of_check_cleaned']

In [11]:
def detect_whale(
        items_per_person:float, 
        price_per_person:float, 
        items_per_person_75th_pctile:float, 
        price_per_person_75_pctile:float) -> str:
    if items_per_person > items_per_person_75th_pctile and price_per_person > price_per_person_75_pctile:
        return 'whale'
    if items_per_person > items_per_person_75th_pctile:
        return 'big eater'
    if price_per_person > price_per_person_75_pctile:
        return 'big spender'
    
    return ''

# tests
ppp_75 = checks['price_per_person'].quantile(0.75)
ipp_75 = checks['items_per_person'].quantile(0.75)
print(ppp_75, ipp_75)
assert detect_whale(5, 250, 3, 175) == 'whale'
assert detect_whale(5, 100, 3, 175) == 'big eater'
assert detect_whale(1, 250, 3, 175) == 'big spender'
assert detect_whale(1, 100, 3, 175) == ''


# Apply the detect_whale function to the checks DataFrame
checks['whale'] = checks.apply(lambda row: detect_whale(row['items_per_person'], row['price_per_person'], ipp_75, ppp_75), axis=1)
checks.sample(25)

158.35666666666668 3.0


,check,date,party size,total items on check,total amount of check,gratuity,total_amount_of_check_cleaned,price_per_item,gratuity_cleaned,price_per_person,items_per_person,tip_percentage,whale
15,2386,2024-03-31,5,12,"$1,147.80",$137.74,1147.80,95.65,137.74,229.560000,2.400000,0.120003,big spender
0,2827,2024-05-06,8,12,$415.08,$107.92,415.08,34.59,107.92,51.885000,1.500000,0.259998,
29,4590,2024-05-08,3,5,$220.40,$22.04,220.40,44.08,22.04,73.466667,1.666667,0.100000,
35,2486,2024-01-27,4,13,$569.01,$108.11,569.01,43.77,108.11,142.252500,3.250000,0.189997,big eater
32,1440,2024-11-30,3,8,$589.04,$141.37,589.04,73.63,141.37,196.346667,2.666667,0.240001,big spender
11,3693,2024-01-18,10,20,"$1,820.00",$309.40,1820.00,91.00,309.40,182.000000,2.000000,0.170000,big spender
12,4528,2024-01-02,4,6,$49.98,$7.50,49.98,8.33,7.50,12.495000,1.500000,0.150060,
40,2512,2024-03-30,3,12,$181.56,$39.94,181.56,15.13,39.94,60.520000,4.000000,0.219982,big eater
31,1945,2024-02-05,3,7,$132.86,$21.26,132.86,18.98,21.26,44.286667,2.333333,0.160018,
37,4829,2024-12-30,9,11,$816.20,$16.32,816.20,74.20,16.32,90.688889,1.222222,0.019995,


## Challenge 3-4-2

#### Write and test your KPI's!!!

In module `check_functions.py`
1. copy over the `detect_whale()` function and tests
2. write function `detect_tipper(tip_pct, tip_pcy_75th_pctile, tip_pct_25_pctile)`
    - should return either "light", "heavy" or ""
3. write tests for `detect_tipper()`

in `3-4-2.py`
1. copy the code from `3-4-1.py`
2. Calculate the ntiles using `.quantile()`
3. call the `apply()` function on the row to make new colums `whale` and `tipper`
 

## Looping over Dataframes

If you must for loop over your DataFrames, there are two choices:

- `df.iterrows()` dict-like iteration
- `df.itertuples()` named-tuple like iteration (faster)


Let's do an example whewre we display the check number, whale and tipper for "heavy tipper" checks.

In [12]:
## Using the iterrows() method
print("Total Amount of Whale Checks")
for i,row in checks.iterrows():
    if row['whale'] == 'whale':
        print(i, row['check'], row['total_amount_of_check_cleaned'])

Total Amount of Whale Checks
16 3694 1574.37
24 4310 3262.3


In [13]:
# Same example with the itertuples() method
print("Total Amount of Whale Checks")
for row in checks.itertuples():
    if row.whale == 'whale':
        print(row.check, row.total_amount_of_check_cleaned)

Total Amount of Whale Checks
3694 1574.37
4310 3262.3


In [14]:
# Of course you don't need a loop to do this:
checks[checks['whale'] == 'whale'][['check', 'total_amount_of_check_cleaned']]

,check,total_amount_of_check_cleaned
16,3694,1574.37
24,4310,3262.30
